# Setup

In [1]:
import os
import openai
import pandas as pd

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

llm_model = "gpt-3.5-turbo"

# LLMChain

This lesson is all about "chaining" input and output from an LLM to get a more sophisticated output from the LLM.

As we'll see, chains can be composed together through other chains to build up processes.

To start with we have the simplest type of chain, which functions like the `chat` feature we've been using so far.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [3]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

C:\Users\hjfen\AppData\Local\Temp\ipykernel_22332\3225343058.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.9, model=llm_model)


In [4]:
prompt_for_llm_chain = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [5]:
llm_chain = LLMChain(llm=llm, prompt=prompt_for_llm_chain)

C:\Users\hjfen\AppData\Local\Temp\ipykernel_22332\3386806982.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_for_llm_chain)


In [6]:
product = "Queen Size Sheet Set"
llm_chain.run(product)

C:\Users\hjfen\AppData\Local\Temp\ipykernel_22332\1011596546.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_chain.run(product)


'Regal Comfort Bedding Co.'

# SimpleSequentialChain

We can link these chains together one after another in a SimpleSequentialChain. The output of one chain becomes the input to the next.

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
# prompt template 1
first_prompt_for_simple_seq = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one_for_simple_seq = LLMChain(llm=llm, prompt=first_prompt_for_simple_seq)

In [9]:
# prompt template 2
second_prompt_for_simple_seq = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two_for_simple_seq = LLMChain(llm=llm, prompt=second_prompt_for_simple_seq)

In [10]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one_for_simple_seq, chain_two_for_simple_seq],
                                             verbose=True
                                            )

In [11]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royalty Linens"
"Royalty Linens provides luxurious and high-quality bed linens fit for royalty, offering a touch of elegance and comfort."

> Finished chain.


'"Royalty Linens provides luxurious and high-quality bed linens fit for royalty, offering a touch of elegance and comfort."'

# SequentialChain

A sequential chain is like the simple version above but several data flow paths through the chain may exists, like so:
                Chain 2 -> Chain 4 -> Output
                /           /
Input -> Chain 1 -> Chain 3

You can design much more complicated prompt flows this way and break a task into subtasks for the llm to solve with separate calls. Can aggregate multiple outputs back together with a prompt.

Use the `input_key` and `output_key` parameters to hook the chains together. The input_keys are inferred from the prompt by lang chain.

In [12]:
from langchain.chains import SequentialChain

In [13]:
# prompt template 1: translate to english
first_prompt_for_seq = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one_for_seq = LLMChain(llm=llm, prompt=first_prompt_for_seq, 
                     output_key="English_Review"
                    )


In [14]:
second_prompt_for_seq = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two_for_seq = LLMChain(llm=llm, prompt=second_prompt_for_seq, 
                     output_key="summary"
                    )

In [15]:
# prompt template 3: translate to english
third_prompt_for_seq = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three_for_seq = LLMChain(llm=llm, prompt=third_prompt_for_seq,
                       output_key="language"
                      )

In [16]:
# prompt template 4: follow up message
fourth_prompt_for_seq = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four_for_seq = LLMChain(llm=llm, prompt=fourth_prompt_for_seq,
                      output_key="followup_message"
                     )


In [17]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_seq_chain = SequentialChain(
    chains=[chain_one_for_seq, chain_two_for_seq, chain_three_for_seq, chain_four_for_seq], # the order of these chains matters, the input variables of the latter chains must be among the output variables of the previous chains
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [18]:
df = pd.read_csv("reviews.csv", delimiter="|")

In [19]:
review = df.Review[5]
overall_seq_chain(review)



> Entering new SequentialChain chain...


C:\Users\hjfen\AppData\Local\Temp\ipykernel_22332\2919783103.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_seq_chain(review)



> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste of the product, noting that the foam does not hold and questioning if it is an old batch or counterfeit.',
 'followup_message': "Cher client,\n\nNous sommes désolés d'apprendre que vous n'avez pas apprécié le goût de notre produit. Nous vous assurons que nous mettons un point d'honneur à fournir des produits de la plus haute qualité à nos clients. Nous allons immédiatement enquêter sur le lot en question pour nous assurer qu'il n'y a aucun problème de qualité. Merci de nous avoir informés de cette situation et nous espérons regagner votre confiance dans le futur.\n\nCordialement,\nL'é

# Router Chain

A Router chain uses an LLM to assess if input is related to other provided chanins and then routs the input accordingly.

In [20]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [21]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [22]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [23]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [24]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [25]:
# The router needs a default route in case it cannot match the provided destinations
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [26]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\hjfen\AppData\Local\Temp\ipykernel_22332\3710404048.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [27]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), # This Parser is what helps the chain route the input
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [28]:
overall_router_chain = MultiPromptChain(router_chain=router_chain, 
                                        destination_chains=destination_chains, 
                                        default_chain=default_chain, verbose=True
                                        )

C:\Users\hjfen\AppData\Local\Temp\ipykernel_22332\2772648066.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  overall_router_chain = MultiPromptChain(router_chain=router_chain,


In [29]:
overall_router_chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the theoretical idealized perfect absorber and emitter of electromagnetic radiation. A black body absorbs all radiation that falls on it and emits radiation across the entire spectrum. The spectrum of radiation emitted by a black body is continuous and depends only on its temperature. This concept is important in understanding the behavior of objects in thermal equilibrium, such as stars.'

In [30]:
overall_router_chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 equals 4.'

In [31]:
overall_router_chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because it is the molecule that carries the genetic instructions for the development, functioning, growth, and reproduction of all living organisms. DNA contains the information needed for building and maintaining an organism, and it is passed down from parent to offspring through reproduction. Having DNA in every cell ensures that each cell has the necessary instructions for carrying out its specific functions and maintaining the overall health and integrity of the organism.'